In [4]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.decomposition import PCA
import utils

In [2]:
# prefered magic functions for jupyter notebook
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import matplotlib
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 

In [14]:
zarr_files = utils.find_files(root_dir = '/root/capsule/data', endswith='zarr')

In [15]:
zarr_files

['/root/capsule/data/me_test1/721291_Face_21707174-ce69-4ce2-b7b5-9e0ce26bbc59/motion_energy_frames.zarr',
 '/root/capsule/data/me_test1/721291_Face_ff11a262-7cbd-4576-8548-1558d6bda9d1/motion_energy_frames.zarr',
 '/root/capsule/data/me_test1/721291_Face_d33cb1d6-b65b-4d2f-8fbd-e93136fd44ae/motion_energy_frames.zarr']

In [18]:
pkl_files = utils.find_files(root_dir = '/root/capsule/data', endswith='.pkl', return_dir=False)

In [19]:
pkl_files

['/root/capsule/data/me_test1/721291_Face_21707174-ce69-4ce2-b7b5-9e0ce26bbc59/721291_Face_21707174-ce69-4ce2-b7b5-9e0ce26bbc59.pkl',
 '/root/capsule/data/me_test1/721291_Face_ff11a262-7cbd-4576-8548-1558d6bda9d1/721291_Face_ff11a262-7cbd-4576-8548-1558d6bda9d1.pkl',
 '/root/capsule/data/me_test1/721291_Face_d33cb1d6-b65b-4d2f-8fbd-e93136fd44ae/721291_Face_d33cb1d6-b65b-4d2f-8fbd-e93136fd44ae.pkl']

In [23]:
import zarr
import dask
import dask.array as da
me_store = zarr.DirectoryStore(zarr_files[0])
me_frames = da.from_zarr(me_store, component='data')

In [24]:
me_frames

dask.array<from-zarr, shape=(272033, 492, 658), dtype=float32, chunksize=(100, 492, 658), chunktype=numpy.ndarray>